In [2]:
import requests
from geopandas import GeoSeries
from shapely.geometry import Polygon
import os
import fnmatch
import os
import rasterio
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.mask import mask
import numpy as np
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html

## Retrieve building coordinates

In [3]:
def get_address():
    
    import requests
    import geopandas
    from geopandas import GeoSeries
    from shapely.geometry import Polygon
    
    street = 'Thonetlaan'
    nb = 133
    pc = 2050
    city = 'Antwerpen'
    
    #Ask for user to input address
    #nb = input("Enter house number:")
    #street = input("Enter street:")
    #city = input("Enter city:")
    #pc = input("Enter postcode:")
    #Example address
    #Eugeen de Bocklaan 14, 2900 Schoten
    
    global building_address
    building_address = (street, str(nb), city, str(pc))
    building_address = " ".join(building_address)
    
    #Check user adddress match using api
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
    
    #Retrieve objectID for users address
    objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]
    
    #Get building geometry
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
    
    objectId = req["gebouw"]["objectId"]

    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()

    
    #Get building polygon coordinates
    global polygon
    polygon = [req["geometriePolygoon"]["polygon"]]
    
    #Convert polygon to more useful geopanda series
    global t
    t = []
    
    #Get coordinates
    for i in polygon[0]['coordinates'][0]:
        t.append(tuple(i))
    
    #Convert coordinates to Polygon format
    global p
    p = Polygon(t)
    
    #Save Polygon in geopanda series
    global g
    g = GeoSeries([p])
    
    #Get area of building
    global a
    #Area of the building
    a = g.area

    return p, a, t

get_address()

(<shapely.geometry.polygon.Polygon at 0x7fbd7360f090>,
 0    1925.465959
 dtype: float64,
 [(151749.25473245978, 213251.19298200682),
  (151725.776012443, 213257.63144601136),
  (151723.63444444537, 213249.82191000506),
  (151723.5725564435, 213249.59605400637),
  (151696.95994842798, 213256.8939740099),
  (151691.53012441844, 213237.093845997),
  (151686.5867644176, 213238.44936599582),
  (151685.4525564164, 213234.31317399442),
  (151716.9713404402, 213225.6698459871),
  (151713.55892443657, 213213.22600597888),
  (151725.2485884428, 213210.02037397772),
  (151722.77185244113, 213200.9886299707),
  (151724.8601084426, 213200.4159579724),
  (151724.53358044475, 213199.22510996833),
  (151732.99546845257, 213196.90459796786),
  (151739.08846045285, 213219.12334998325),
  (151740.36436445266, 213218.7734619826),
  (151749.25473245978, 213251.19298200682)])

## Create list of all DSM and DTM raster files

In [60]:
def raster_list():
    #Create list of all raster files
    import os
    import fnmatch
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
    folder_path = path
    
    global dsmlist, dtmlist
    dsmlist = []
    dtmlist = []


    for path, dirs, files in os.walk(folder_path):
            for filename in files:
                if fnmatch.fnmatch(filename, '*DSM*'):
                    dsmlist.append(filename)
                if fnmatch.fnmatch(filename, '*DTM*'):
                    dtmlist.append(filename)
            print("There are",len(dsmlist), "DSM raster files.")
            print("There are",len(dtmlist), "DTM raster files.")
            
raster_list()

There are 43 DSM raster files.
There are 43 DTM raster files.


## Get bounds of all raster files in Flanders

In [65]:
def get_bounds():

    #Get bounds of rasters
    import os
    import rasterio
    import pandas as pd

    #Set path
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')

    #Make an empty lists
    dsmbounds = []
    dsmfilenames = []

    for f in dsmlist:
    
        filepath = os.path.join(path, f)
    
        #Add raster filename to list
        dsmfilenames.append(f)
    
        #Open raster and add bounds to list
        with rasterio.open(filepath) as src:
            dsmbounds.append(src.bounds)
            #rasterio.features.bounds(src)
    
    #Create df of filenames and bounds
    global dsm_df
    dsm_df = pd.DataFrame({'filelist': dsmfilenames,
                   'bounds': dsmbounds})
 
    #Write df to a file
    dsm_df.to_csv("dsmbounds.csv", index=False)

    #Make an empty lists
    dtmbounds = []
    dtmfilenames = []
    
    for f in dtmlist:
    
        filepath = os.path.join(path, f)
    
        #Add raster filename to list
        dtmfilenames.append(f)
    
        #Open raster and add bounds to list
        with rasterio.open(filepath) as src:
            dtmbounds.append(src.bounds)
            if rasterio.coords.disjoint_bounds(src.bounds, p.bounds) == False:
                print(src.name)
             
    #Create df of filenames and bounds
    global dtm_df
    dtm_df = pd.DataFrame({'dsmfile': dsmfilenames, 'dtmfile': dtmfilenames,
                   'bounds': dtmbounds})
 
    #Write df to a file
    dtm_df.to_csv("dtmbounds.csv", index=False)
    
get_bounds()

/media/becode/GOPRO2/1GEOTIFF/DHMVIIDTMRAS1m_k15.tif


In [57]:
file = df.dsmfile[1]

 #Set path
path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')

filepath = os.path.join(path, file)
    
with rasterio.open(filepath) as src:
    m = src.bounds

In [59]:
rasterio.coords.disjoint_bounds(m, p.bounds)

True

In [42]:
df.dsmfile[1]

rasterio.coords.disjoint_bounds(m, p.bounds)

'DHMVIIDSMRAS1m_k01.tif'

In [337]:
r1= dtm_df.bounds[0][:]
r2 = dtm_df.bounds[1][:]
r3 = dtm_df.bounds[2][:]
r4 = dtm_df.bounds[3][:]
r5 = dtm_df.bounds[4][:]
r6= dtm_df.bounds[5][:]
r7= dtm_df.bounds[6][:]
r8= dtm_df.bounds[7][:]
r9= dtm_df.bounds[8][:]
r10 = dtm_df.bounds[9][:]
r11 = dtm_df.bounds[10][:]
r12 = dtm_df.bounds[11][:]
r13 = dtm_df.bounds[12][:]
r14 = dtm_df.bounds[13][:]
r15 = dtm_df.bounds[14][:]
r16 = dtm_df.bounds[15][:]
r17 = dtm_df.bounds[16][:]
r18 = dtm_df.bounds[17][:]
r19 = dtm_df.bounds[18][:]
r20 = dtm_df.bounds[19][:]
r21 = dtm_df.bounds[20][:]
r22 = dtm_df.bounds[21][:]
r23 = dtm_df.bounds[22][:]
r24 = dtm_df.bounds[23][:]
r25 = dtm_df.bounds[24][:]
r26 = dtm_df.bounds[25][:]
r27 = dtm_df.bounds[26][:]
r28 = dtm_df.bounds[27][:]
r29 = dtm_df.bounds[28][:]
r30 = dtm_df.bounds[29][:]
r31 = dtm_df.bounds[30][:]
r32 = dtm_df.bounds[31][:]
r33 = dtm_df.bounds[32][:]
r34 = dtm_df.bounds[33][:]
r35 = dtm_df.bounds[34][:]
r36 = dtm_df.bounds[35][:]
r37 = dtm_df.bounds[36][:]
r38 = dtm_df.bounds[37][:]
r39 = dtm_df.bounds[38][:]
r40 = dtm_df.bounds[39][:]
r41 = dtm_df.bounds[40][:]
r42 = dtm_df.bounds[41][:]
r43 = dtm_df.bounds[42][:]



In [342]:
dtm_df['bounds'] = l
dtm_df
dtm_df.to_csv("dtmbounds.csv", index=False)

In [338]:
l = [
    r1,
    r2,
    r3,
    r4,
    r5,
    r6,
    r7,
    r8,
    r9,
    r10,
    r11,
    r12,
    r13,
    r14,
    r15,
    r16,
    r17,
    r18,
    r19,
    r20,
    r21,
    r22,
    r23,
    r24,
    r25,
    r26,
    r27,
    r28,
    r29,
    r30,
    r31,
    r32,
    r33,
    r34,
    r35,
    r36,
    r37,
    r38,
    r39,
    r40,
    r41,
    r42,
    r43
]
l

[(130000.0, 198000.0, 162000.0, 218000.0),
 (145000.0, 238000.0, 162000.0, 247000.0),
 (162000.0, 238000.0, 194000.0, 250000.0),
 (194000.0, 238000.0, 206000.0, 248000.0),
 (53000.0, 218000.0, 66000.0, 225500.0),
 (66000.0, 218000.0, 98000.0, 232000.0),
 (98000.0, 218000.0, 130000.0, 227000.0),
 (130000.0, 218000.0, 162000.0, 238000.0),
 (162000.0, 218000.0, 194000.0, 238000.0),
 (194000.0, 218000.0, 226000.0, 238000.0),
 (226000.0, 218000.0, 238000.0, 227000.0),
 (18500.0, 198000.0, 34000.0, 206500.0),
 (34000.0, 198000.0, 66000.0, 218000.0),
 (66000.0, 198000.0, 98000.0, 218000.0),
 (98000.0, 198000.0, 130000.0, 218000.0),
 (162000.0, 198000.0, 194000.0, 218000.0),
 (194000.0, 198000.0, 226000.0, 218000.0),
 (226000.0, 198000.0, 258000.0, 218000.0),
 (17000.0, 178000.0, 34000.0, 198000.0),
 (34000.0, 178000.0, 66000.0, 198000.0),
 (66000.0, 178000.0, 98000.0, 198000.0),
 (98000.0, 178000.0, 130000.0, 198000.0),
 (130000.0, 178000.0, 162000.0, 198000.0),
 (162000.0, 178000.0, 194000.0

In [253]:
dtm_df.newcol.to_csv("d.csv", index=False) 

In [339]:
dtm_df

,dsmfile,dtmfile,bounds
0,DHMVIIDSMRAS1m_k15.tif,DHMVIIDTMRAS1m_k15.tif,"(130000.0, 198000.0, 162000.0, 218000.0)"
1,DHMVIIDSMRAS1m_k01.tif,DHMVIIDTMRAS1m_k01.tif,"(145000.0, 238000.0, 162000.0, 247000.0)"
2,DHMVIIDSMRAS1m_k02.tif,DHMVIIDTMRAS1m_k02.tif,"(162000.0, 238000.0, 194000.0, 250000.0)"
3,DHMVIIDSMRAS1m_k03.tif,DHMVIIDTMRAS1m_k03.tif,"(194000.0, 238000.0, 206000.0, 248000.0)"
4,DHMVIIDSMRAS1m_k04.tif,DHMVIIDTMRAS1m_k04.tif,"(53000.0, 218000.0, 66000.0, 225500.0)"
5,DHMVIIDSMRAS1m_k05.tif,DHMVIIDTMRAS1m_k05.tif,"(66000.0, 218000.0, 98000.0, 232000.0)"
6,DHMVIIDSMRAS1m_k06.tif,DHMVIIDTMRAS1m_k06.tif,"(98000.0, 218000.0, 130000.0, 227000.0)"
7,DHMVIIDSMRAS1m_k13.tif,DHMVIIDTMRAS1m_k07.tif,"(130000.0, 218000.0, 162000.0, 238000.0)"
8,DHMVIIDSMRAS1m_k09.tif,DHMVIIDTMRAS1m_k08.tif,"(162000.0, 218000.0, 194000.0, 238000.0)"
9,DHMVIIDSMRAS1m_k11.tif,DHMVIIDTMRAS1m_k09.tif,"(194000.0, 218000.0, 226000.0, 238000.0)"


In [304]:
regex='(?<=\=)(.*?)(?=\,|\))'
#df[['left', 'bottom', 'right','top']] = 
c= dtm_df['bounds'].str.split(regex, expand=True)
#(c[1],c[3],c[5],c[7])

c

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [300]:
bb = []
for i in dtm_df.bounds:
    r = dtm_df.bounds[i][:]
    bb.append(r)
bb

KeyError: BoundingBox(left=130000.0, bottom=198000.0, right=162000.0, top=218000.0)

In [287]:
dtm_df.bounds

0     (130000.0, 198000.0, 162000.0, 218000.0)
1     (145000.0, 238000.0, 162000.0, 247000.0)
2     (162000.0, 238000.0, 194000.0, 250000.0)
3     (194000.0, 238000.0, 206000.0, 248000.0)
4       (53000.0, 218000.0, 66000.0, 225500.0)
5       (66000.0, 218000.0, 98000.0, 232000.0)
6      (98000.0, 218000.0, 130000.0, 227000.0)
7     (130000.0, 218000.0, 162000.0, 238000.0)
8     (162000.0, 218000.0, 194000.0, 238000.0)
9     (194000.0, 218000.0, 226000.0, 238000.0)
10    (226000.0, 218000.0, 238000.0, 227000.0)
11      (18500.0, 198000.0, 34000.0, 206500.0)
12      (34000.0, 198000.0, 66000.0, 218000.0)
13      (66000.0, 198000.0, 98000.0, 218000.0)
14     (98000.0, 198000.0, 130000.0, 218000.0)
15    (162000.0, 198000.0, 194000.0, 218000.0)
16    (194000.0, 198000.0, 226000.0, 218000.0)
17    (226000.0, 198000.0, 258000.0, 218000.0)
18      (17000.0, 178000.0, 34000.0, 198000.0)
19      (34000.0, 178000.0, 66000.0, 198000.0)
20      (66000.0, 178000.0, 98000.0, 198000.0)
21     (98000

In [277]:
bb = []

for i in c:
    w = (c[1][i],c[3][i],c[5][i],c[7][i])
    bb.append(w)
bb

KeyError: 1

In [268]:
c[1]

0     130000.0
1     145000.0
2     162000.0
3     194000.0
4      53000.0
5      66000.0
6      98000.0
7     130000.0
8     162000.0
9     194000.0
10    226000.0
11     18500.0
12     34000.0
13     66000.0
14     98000.0
15    162000.0
16    194000.0
17    226000.0
18     17000.0
19     34000.0
20     66000.0
21     98000.0
22    130000.0
23    162000.0
24    194000.0
25    226000.0
26     20000.0
27     34000.0
28     66000.0
29     98000.0
30    130000.0
31    162000.0
32    194000.0
33    226000.0
34    258000.0
35     34000.0
36     66000.0
37     98000.0
38    130000.0
39    162000.0
40    194000.0
41    226000.0
42    258000.0
Name: 1, dtype: object

In [216]:
n = new.str.split('=', expand=True

AttributeError: 'DataFrame' object has no attribute 'str'

In [45]:
qq = pd.DataFrame({'filelist': dsm_df.filelist, 'bounds': dsm_df.bounds})

In [86]:
dtm_df.dtypes

filelist    object
bounds      object
dtype: object

In [173]:
#df = pd.read_csv("dsmbounds.csv")
#dsm_df = pd.DataFrame({'filelist': df.filelist, 'bounds': df.bounds})
df = pd.read_csv("dtmbounds.csv")
dtm = pd.DataFrame({'filelist': df.filelist, 'bounds': df.bounds})

ValueError: too many values to unpack (expected 4)

In [8]:
#dsm_df = pd.read_csv("dsmbounds.csv")
#dsm_df = pd.DataFrame({'filelist': df.filelist, 'bounds': df.bounds})
df = pd.read_csv("dtmbounds.csv")
#dtm = pd.DataFrame({'filelist': df.filelist, 'bounds': df.bounds})
df

,dsmfile,dtmfile,bounds
0,DHMVIIDSMRAS1m_k15.tif,DHMVIIDTMRAS1m_k15.tif,"(130000.0, 198000.0, 162000.0, 218000.0)"
1,DHMVIIDSMRAS1m_k01.tif,DHMVIIDTMRAS1m_k01.tif,"(145000.0, 238000.0, 162000.0, 247000.0)"
2,DHMVIIDSMRAS1m_k02.tif,DHMVIIDTMRAS1m_k02.tif,"(162000.0, 238000.0, 194000.0, 250000.0)"
3,DHMVIIDSMRAS1m_k03.tif,DHMVIIDTMRAS1m_k03.tif,"(194000.0, 238000.0, 206000.0, 248000.0)"
4,DHMVIIDSMRAS1m_k04.tif,DHMVIIDTMRAS1m_k04.tif,"(53000.0, 218000.0, 66000.0, 225500.0)"
5,DHMVIIDSMRAS1m_k05.tif,DHMVIIDTMRAS1m_k05.tif,"(66000.0, 218000.0, 98000.0, 232000.0)"
6,DHMVIIDSMRAS1m_k06.tif,DHMVIIDTMRAS1m_k06.tif,"(98000.0, 218000.0, 130000.0, 227000.0)"
7,DHMVIIDSMRAS1m_k13.tif,DHMVIIDTMRAS1m_k07.tif,"(130000.0, 218000.0, 162000.0, 238000.0)"
8,DHMVIIDSMRAS1m_k09.tif,DHMVIIDTMRAS1m_k08.tif,"(162000.0, 218000.0, 194000.0, 238000.0)"
9,DHMVIIDSMRAS1m_k11.tif,DHMVIIDTMRAS1m_k09.tif,"(194000.0, 218000.0, 226000.0, 238000.0)"


In [16]:
df = pd.read_csv("bounds.csv")
dsm_df = df[['dsmfile', 'bounds']]
dtm_df = df[['dtmfile', 'bounds']]


In [25]:
#Check which DTM rasters overlap the building boundary box
dtmlist=[]

for i in dtm_df['bounds']:
    dtmlist.append(rasterio.coords.disjoint_bounds(i, p.bounds))

ValueError: Bounds must both have the same orientation

In [29]:
dir(rasterio.coords.disjoint_bounds)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [36]:
rasterio.coords.disjoint_bounds

AttributeError: 'function' object has no attribute 'view'

In [26]:
print(p.bounds)
print(dtm_df.bounds)

(151685.4525564164, 213196.90459796786, 151749.25473245978, 213257.63144601136)
0     (130000.0, 198000.0, 162000.0, 218000.0)
1     (145000.0, 238000.0, 162000.0, 247000.0)
2     (162000.0, 238000.0, 194000.0, 250000.0)
3     (194000.0, 238000.0, 206000.0, 248000.0)
4       (53000.0, 218000.0, 66000.0, 225500.0)
5       (66000.0, 218000.0, 98000.0, 232000.0)
6      (98000.0, 218000.0, 130000.0, 227000.0)
7     (130000.0, 218000.0, 162000.0, 238000.0)
8     (162000.0, 218000.0, 194000.0, 238000.0)
9     (194000.0, 218000.0, 226000.0, 238000.0)
10    (226000.0, 218000.0, 238000.0, 227000.0)
11      (18500.0, 198000.0, 34000.0, 206500.0)
12      (34000.0, 198000.0, 66000.0, 218000.0)
13      (66000.0, 198000.0, 98000.0, 218000.0)
14     (98000.0, 198000.0, 130000.0, 218000.0)
15    (162000.0, 198000.0, 194000.0, 218000.0)
16    (194000.0, 198000.0, 226000.0, 218000.0)
17    (226000.0, 198000.0, 258000.0, 218000.0)
18      (17000.0, 178000.0, 34000.0, 198000.0)
19      (34000.0, 178000.0,

In [131]:
# Get Bounding Box of the entered address
            bb_addr = BBox(req["LocationResult"][0]["BoundingBox"]["LowerLeft"]["X_Lambert72"],
                           req["LocationResult"][0]["BoundingBox"]["LowerLeft"]["Y_Lambert72"],
                           req["LocationResult"][0]["BoundingBox"]["UpperRight"]["X_Lambert72"],
                           req["LocationResult"][0]["BoundingBox"]["UpperRight"]["Y_Lambert72"])
    
pp = Polygon(polygon)
#rasterio.features.bounds(pp, north_up=True)

ValueError: A LinearRing must have at least 3 coordinate tuples

In [135]:
Polygon(polygon)

ValueError: A LinearRing must have at least 3 coordinate tuples

In [138]:
coords = np.asarray(polygon)
coords
rasterio.features.bounds(coords, north_up=True)

AttributeError: 'numpy.ndarray' object has no attribute 'get'

## Search for raster file that contains the user address

In [95]:
def raster_overlap():

    import geopandas as gpd
    import rasterio
    
    ##Get filenames of overlapping rasters
    dsmlist = []
    dtmlist = []
    
    #Open DSM and DTM raster bounds as df
    #dsm_df = gpd.read_file('dsmbounds.csv')
    #dtm_df = gpd.read_file('dtmbounds.csv')
    #reading bounds from csv gives an error
    
    #Check which DSM rasters overlap the building boundary box
    for i in dsm_df['bounds']:
        dsmlist.append(rasterio.coords.disjoint_bounds(i, p.bounds))

    #Reverse the booleen
    boo = [not bool for bool in dsmlist]

    #Keep the dsm filename
    global dsmfile
    dsmfile = dsm_df.filelist[boo].item()

    #Check which DTM rasters overlap the building boundary box
    for i in dtm_df['bounds']:
        dtmlist.append(rasterio.coords.disjoint_bounds(i, p.bounds))

    #Reverse the booleen
    boo = [not bool for bool in dsmlist]

    #Keep the dtm filename
    global dtmfile
    dtmfile = dtm_df.filelist[boo].item()

    return dsmfile, dtmfile

raster_overlap()

ValueError: Bounds must both have the same orientation

In [115]:
print(dsm_df.bounds[1])

p.bounds

BoundingBox(left=145000.0, bottom=238000.0, right=162000.0, top=247000.0)


(151685.4525564164, 213196.90459796786, 151749.25473245978, 213257.63144601136)

## Calculate Digital Elevation Model for building

In [6]:
def calculate_dem():
    
    import os
    import rasterio
    from rasterio.windows import Window
    from rasterio.mask import mask
    import numpy as np
    import plotly.graph_objects as go

    #File paths required
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
    dsmpath = os.path.join(path, str(dsmfile))
    dtmpath = os.path.join(path, str(dtmfile))

    #Open DSM raster with mask of building shape
    with rasterio.open(dsmpath) as src:
        mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
        #Read only pixels within the window/bounds of the building shape
        dsm = src.read(1, window = win)

    #Open DTM raster with mask of building shape
    with rasterio.open(dtmpath) as src:
        mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
        #Read only pixels within the window/bounds of the building shape
        dtm = src.read(1, window = win)

    #Calculates raw digital elevation model (no resampling)
    global dem
    dem = dsm - dtm

#calculate_dem()

## Get the height and floor area of the building

In [7]:
def get_height():
    #Show building height and floor area
    height = round(dem.max(), 1)
    print('The building height is:', height, 'meters')

def get_area():
    area = round(int(a), 1)
    print('The building floor area is:', area, 'sq meters')
    
#get_height()
#get_area()

## Plot the building

In [8]:
def plot_building():
    import plotly.graph_objects as go

    #Plot xyz of building
    fig = go.Figure(data=[go.Surface(z=dem)])

    fig.update_layout(title=str(building_address), autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()

    import dash
    import dash_core_components as dcc
    import dash_html_components as html

    app = dash.Dash()
    app.layout = html.Div([
    dcc.Graph(figure=fig)
    ])

    app.run_server(debug=True, use_reloader=False)

#plot_building()

## Make 3D house

In [9]:
#Make 3D house
get_address()
raster_overlap()
calculate_dem()
get_height()
get_area()
plot_building()

Enter house number:133
Enter street:thonetlaan
Enter city:antwerpen
Enter postcode:2050


NameError: name 'dsm_df' is not defined